In [ ]:
# t-SNE http://www.scikit-yb.org/en/latest/api/text/tsne.html

from yellowbrick.text import TSNEVisualizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import HashingVectorizer
import codecs

class Corpus(object):
    def __init__(self, sentences):
        self.documents = codecs.open(sentences, encoding="utf-8").readlines()
        self.document_list = []
        for d in self.documents:
            self.document_list.append([d])

def load_corpus():
    c = Corpus("all_posts01.txt")
    return c
    

corpus = load_corpus()


In [ ]:
#tfidf  = TfidfVectorizer(stop_words='english')
from sklearn.cluster import KMeans
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                                 min_df=2,
                                 use_idf=True)
#transformer =  TfidfTransformer()
#tfidf = make_pipeline(hasher,transformer)
docs   = vectorizer.fit_transform(corpus.documents)

print(docs)

true_k = 500
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(docs)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

#for i in range(true_k):
    #print("Cluster %d:" % i)
    #for ind in order_centroids[i, :10]:
        #print(' %s' % terms[ind])
        

In [ ]:
tsne = TSNEVisualizer(labels=["documents"])
tsne.fit(docs)
tsne.poof()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import IncrementalPCA, FastICA
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean
import numpy as np

stri = 'Any girl being topless hey'
response = vectorizer.transform([stri])
r = [((i, j), response[i,j]) for i, j in zip(*response.nonzero())]
for entry in r:
    word = terms[entry[0][1]]
    confidence = entry[1]
    print(word)
    print(float(confidence))


In [89]:
import gensim
import codecs
# used for loading or saving
model_file = '/home/marcel/projects/phaedra/models/xxxchatters_posts05_-min20chars_20epochs_lower.d2v'
filename = '/home/marcel/projects/phaedra/xxxchatters_posts05.txt'

sentences = []
from random import shuffle

counter = 0
for uid, line in enumerate(codecs.open(filename, encoding='utf-8')):
    if len(line) > 20:
        ls = gensim.models.doc2vec.LabeledSentence(words=line.lower().split(), tags=['SENT_%s' % counter])
        sentences.append(ls)
        counter += 1
print(len(sentences),'sentences')


# Training the doc2vec model

# tutorial https://rare-technologies.com/doc2vec-tutorial/
# proposes shuffling or learning reate adjustment. we gonna do both
# in total 20 epochs
model = gensim.models.Doc2Vec(size=100, window=8, min_count=2, workers=4)  # use fixed learning rate
print('building vocab') 
model.build_vocab(sentences)

base_alpha = model.alpha
base_min_alpha = model.min_alpha

for mepoch in range(2):
    model.alpha = base_alpha 
    model.min_alpha = base_min_alpha
    for epoch in range(10):
        print('epoch',mepoch * 10 + epoch)
        model.train(sentences, total_examples=model.corpus_count, epochs=1)
        model.alpha -= 0.002  # decrease the learning rate
        model.min_alpha = model.alpha  # fix the learning rate, no decay
    #shuffle(sentences)

# saving the model    
model.save(model_file)



61625 sentences
building vocab
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19


In [29]:
import gensim
import codecs
# used for loading or saving
#model_file = '/home/marcel/projects/phaedra/models/all_posts01.doc2vec'

# Tiny helper
def nice_print(tagged_doc):
    return (' '.join(sentences[int(tagged_doc[0][5:])][0]))

# Build sentence list (each sentence needs at least 1 tag)
#filename = '/home/marcel/projects/phaedra/all_posts01.txt'

#model_file = '/home/marcel/projects/phaedra/models/xxxchatters_longer_training-min15chars2.d2v'
#filename = '/home/marcel/projects/phaedra/xxxchatters_posts.txt'

#sentences = []
#from random import shuffle

#for uid, line in enumerate(codecs.open(filename, encoding='utf-8')):
#    if len(line) > 15:
#        ls = gensim.models.doc2vec.LabeledSentence(words=line.split(), tags=['SENT_%s' % uid])
#        sentences.append(ls)
#print(len(sentences),'sentences')

#model_loaded = gensim.models.Doc2Vec.load(model_file)
model_loaded = model

number = 201
print(' '.join(sentences[number][0]))
sims = model_loaded.docvecs.most_similar('SENT_'+str(number),topn=20)
print('similar sentence',len(sims))
print('\nSIMILAR SENTENCES\n')
for sim in sims:
    print(nice_print(sim))






108460 sentences
14 Radio Meltdown: DJ`liltech is playing 6 George Thorogood - Bad To The Bone
similar sentence 20

SIMILAR SENTENCES



IndexError: list index out of range

In [5]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

X = model_loaded[model_loaded.wv.vocab]

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

In [6]:
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], s=0.1)
plt.savefig("out_xxxchatters.pdf")

In [107]:
def get_similar(model, words, top_n):
    inferred_vector = model.infer_vector(words, steps=1000, alpha=0.025)
    sims = model.docvecs.most_similar([inferred_vector], topn=top_n)
    return sims

#sen = "i love it when ladies want top shelf".lower()
#sen = "toilet shit".lower()
sen = "i have to maintain some discretion lol".lower()
print(sen)
result = get_similar(model, sen.split(), 200)
#print(result)
for res in result:
    index = res[0].split('_')[1]
    print(' '.join(sentences[int(index)].words), index)

i have to maintain some discretion lol
kylie: lol i have some tettleys 28827
i have to maintain some discretion lol 819
some reciprocation, please, lol 32834
watching some ofsilfs (olympic figure skater) 60334
i can see some advantages 47594
has anyone seen some lettuce? 60373
we all have our moments lol 25576
some nights more than others ... 52033
but, i have some girth! 10892
no accounting for some tastes. lol 8384
13went dancing with some freinds 44877
some follow that closely. 31295
any females free to consider some rp roles, pm me please 12533
tossed filthywife some soap 25933
the kiwis have some rockin pipers 3587
some would say wastefull 44095
13recycled champayne lol 41046
let me guess....too many? lol 53188
look at all the colors! lol 7552
better..scotsman lol 5187
gotta have some standards. 32551
lol there's some baha men in here 25572
i've honestly never been lol 4607
i have some pizza in the fridge 11217
prepping`pudding: sprung a leak lol 9579
lol i have coconut cookies 

In [110]:
import random
number = random.randint(0, len(sentences))
number = 819
print(' '.join(sentences[number][0]))
sims = model.docvecs.most_similar('SENT_'+str(number),topn=40)
print('\nSIMILAR SENTENCES\n')
for sim in sims:
    print(nice_print(sim))


i have to maintain some discretion lol

SIMILAR SENTENCES

kylie: lol i have some tettleys
any females free to consider some rp roles, pm me please
some places anyways:p
some times... some days
offers katiets some crackers
any ladies fancy some rp?>
some reciprocation, please, lol
12into some wild mmmf?
any ladies fancy some rp?
tossed filthywife some soap
some nights more than others ...
anyone interested in some rp?
doctor howzer recommends some rest. doctor demento recommends some ray stevens
fox you need to learn some manners
the kiwis have some rockin pipers
gotta have some standards.
i can see some advantages
some follow that closely.
watching some ofsilfs (olympic figure skater)
you put on some weight?
12,00at times, some guidance increaases the 'better'
stir fry some veggies
listening some cats yowling
saving some for youuuuuu toooooo
someone for some steamy pv
had a 1918 solera some years ago
some would say wastefull
gigabit in some places
12,00needs lunch, of some sort
yikes,

In [100]:
#print (' '.join(sentences[9][0]))
#sims = model.docvecs.most_similar('SENT_10')
#print('similar sentence',len(sims))
#print('\nSIMILAR SENTENCES\n')
#for sim in sims:
#    print(nice_print(sim))


# Tiny helper
def nice_print(tagged_doc, print_doc):
    if print_doc:
        print(tagged_doc)
    return(' '.join(sentences[int(tagged_doc[0][5:])][0]))